# EDA

Preprocess and analyze datasets from https://data.mos.ru

## Data preparation

In [1]:
from os import chdir, path, getcwd
from pathlib import Path

chdir(Path(path.realpath("__file__")).parents[0].parents[0])
getcwd()

'/Users/ksusonic/mlds/travel-agent'

In [3]:
import pandas as pd

# Театры https://data.mos.ru/opendata/531
theatres = pd.read_json(
    "https://data.mos.ru/odata/export/catalog?idFile=270312",
    compression="zip",
    encoding="windows-1251",
)

# Объекты культурного наследия федерального значения https://data.mos.ru/opendata/62302
cultural = pd.read_json(
    "https://data.mos.ru/odata/export/catalog?idFile=262466",
    compression="zip",
    encoding="windows-1251",
)

In [12]:
from travel_agent.data.theatres import processed_theatres
from travel_agent.data.cultural import processed_cultural

datasets = {
    "theatres": processed_theatres(theatres),
    "cultural": processed_cultural(cultural),
}

for name, dataset in datasets.items():
    dataset["type"] = name

merged = pd.concat([datasets[dataset] for dataset in datasets])
merged.to_csv("data/merged_mosru.csv", index=False)
merged.head()

/Users/ksusonic/mlds/travel-agent/travel_agent/data/theatres.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_theaters["District"] = processed_theaters["ObjectAddress"].map(
/Users/ksusonic/mlds/travel-agent/travel_agent/data/theatres.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_theaters["Address"] = processed_theaters["ObjectAddress"].map(
/Users/ksusonic/mlds/travel-agent/travel_agent/data/theatres.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,Name,geoData,District,Address,type
0,Театр кукол «Жар-птица»,"{'coordinates': [[37.683080444, 55.790769624]]...",район Сокольники,"Москва, внутригородская территория муниципальн...",theatres
1,Творческая мастерская А. Рыбникова,"{'coordinates': [[37.605678057, 55.763454738]]...",Тверской район,"Москва, внутригородская территория муниципальн...",theatres
2,Театр «Мастерская Фоменко»,"{'coordinates': [[37.53775816, 55.741621711], ...",район Дорогомилово,"Москва, внутригородская территория муниципальн...",theatres
3,Московский театр кукол,"{'coordinates': [[37.661539, 55.835015], [37.6...",Басманный район,"Москва, внутригородская территория муниципальн...",theatres
4,Театр «Эрмитаж»,"{'coordinates': [[37.608618, 55.770619], [37.5...",район Арбат,"Москва, внутригородская территория муниципальн...",theatres
